# Loading data

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_excel('vente_maillots_de_bain.xlsx',sheet_name='Sheet2')
df.head()

,Years,Sales,Unnamed: 2
0,2001-01-01,139.7,NaN
1,2001-02-01,114.3,NaN
2,2001-03-01,101.6,NaN
3,2001-04-01,152.4,NaN
4,2001-05-01,215.9,NaN
